In [1]:
# The Usual
import pandas as pd

#NLTK
import nltk; nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy
import pyfpgrowth
# FP-Growth
# from mlxtend import frequent_patterns
# from mlxtend.preprocessing import TransactionEncoder

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\uyent\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\uyent\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
listings = pd.read_csv("listings.csv", usecols=['id', 'name', 'description', 'neighbourhood_cleansed', 'zipcode', 'property_type', 'room_type', 
                                            'price', 'availability_365'])
listings = listings.rename(columns={'id': 'listing_id'})

reviews = pd.read_csv("reviews.csv", usecols=['listing_id', 'comments'])

In [4]:
df = pd.merge(listings, reviews, on='listing_id')

In [5]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [6]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(df['name']))


In [7]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [8]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)


In [9]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# ! python3 -m spacy download en_core_web_sm

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [11]:
# te = TransactionEncoder()
# te_ary = te.fit(data_words_bigrams).transform(data_words_bigrams)
# df = pd.DataFrame(te_ary, columns=te.columns_)
# df

patterns= pyfpgrowth.find_frequent_patterns(data_words_nostops, 8)
patterns

{('utopic',): 8,
 ('bed', 'utopic'): 8,
 ('utopic', 'williamsburg'): 8,
 ('apt', 'utopic'): 8,
 ('bed', 'utopic', 'williamsburg'): 8,
 ('apt', 'bed', 'utopic'): 8,
 ('apt', 'utopic', 'williamsburg'): 8,
 ('apt', 'bed', 'utopic', 'williamsburg'): 8,
 ('basquiat',): 8,
 ('absolutely', 'basquiat'): 8,
 ('basquiat', 'million'): 8,
 ('basquiat', 'gorgeous'): 8,
 ('basquiat', 'loft'): 8,
 ('absolutely', 'basquiat', 'million'): 8,
 ('absolutely', 'basquiat', 'gorgeous'): 8,
 ('absolutely', 'basquiat', 'loft'): 8,
 ('basquiat', 'gorgeous', 'million'): 8,
 ('basquiat', 'loft', 'million'): 8,
 ('basquiat', 'gorgeous', 'loft'): 8,
 ('absolutely', 'basquiat', 'gorgeous', 'million'): 8,
 ('absolutely', 'basquiat', 'loft', 'million'): 8,
 ('absolutely', 'basquiat', 'gorgeous', 'loft'): 8,
 ('basquiat', 'gorgeous', 'loft', 'million'): 8,
 ('absolutely', 'basquiat', 'gorgeous', 'loft', 'million'): 8,
 ('win',): 8,
 ('lgbt', 'win'): 8,
 ('gay', 'win'): 8,
 ('twin', 'win'): 8,
 ('friendly', 'win'): 8,
 

In [ ]:
# frequent_patterns.fpgrowth(df, min_support=0.4, use_colnames=True)